In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer

from googletrans import Translator
from deep_translator import GoogleTranslator
import string
import emoji
from bs4 import BeautifulSoup

from dotenv import load_dotenv
import os

import itertools
from html import unescape
import re 

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
df = pd.read_csv('/Users/alenjose/Desktop/data/project_data/self-scraped/processed/ME_TOO_english_tweets.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28629 entries, 0 to 28628
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              28629 non-null  int64  
 1   tweetid                 28629 non-null  int64  
 2   date                    28629 non-null  object 
 3   tweet_rendered_content  28623 non-null  object 
 4   url                     28629 non-null  object 
 5   latitude                45 non-null     float64
 6   longitude               45 non-null     float64
 7   retweets                28629 non-null  int64  
 8   favorites               28629 non-null  int64  
 9   source.r                23788 non-null  object 
 10  hashtag                 28629 non-null  object 
 11  num.emojis              28629 non-null  int64  
 12  emoji_names             1479 non-null   object 
 13  year                    28629 non-null  int64  
 14  month                   28629 non-null

In [6]:
df.head()

,Unnamed: 0,tweetid,date,tweet_rendered_content,url,latitude,longitude,retweets,favorites,source.r,hashtag,num.emojis,emoji_names,year,month
0,0,1,2017-10-16 19:36:24,wasn going to tweet metoo because weve been tw...,https://twitter.com/_________pia/status/920010...,NaN,NaN,0,0,IPHONE,#metoo,0,NaN,2017,10
1,1,2,2017-10-16 20:05:12,me seeing you post metoo several months after ...,https://twitter.com/___eilatan/status/92001780...,NaN,NaN,0,0,ANDROID,#metoo,0,NaN,2017,10
2,2,3,2017-10-16 21:31:34,me too metoo,https://twitter.com/___Jill_Ryan___/status/920...,NaN,NaN,0,0,NaN,#metoo,0,NaN,2017,10
3,3,4,2017-10-16 23:11:32,all the metoo statuses and there so many peopl...,https://twitter.com/___NS16/status/92006469861...,NaN,NaN,0,1,NaN,#metoo,0,NaN,2017,10
4,4,5,2017-10-16 20:42:16,every wolf whistle every hey baby every forced...,https://twitter.com/___SAAJ___/status/92002713...,NaN,NaN,0,1,ANDROID,#metoo,0,NaN,2017,10


In [7]:
#  drop null
df_train = df[df['tweet_rendered_content'].notnull()]

# load model
model_path = '/Users/alenjose/Desktop/data/project_data/model_info/LSTM_with_sw_82_perc.keras'
model = load_model(model_path, compile=False)

# compile model with new optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# preprocess tweets
vocab_size = 7000  # as used during training
oov_tok = '<OOV>'
max_length = 200   # as used during training
padding_type='post'
trunc_type='post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df_train['tweet_rendered_content'])

tweet_sequences = tokenizer.texts_to_sequences(df_train['tweet_rendered_content'])
tweet_padded = pad_sequences(tweet_sequences, padding=padding_type, maxlen=max_length, truncating=trunc_type)

predictions = model.predict(tweet_padded)

sentiments = []

for pred in predictions:
    if pred > 0.5:
        sentiments.append('positive')
    elif pred < 0.5:
        sentiments.append('negative')
    else:
        sentiments.append('neutral')

df_train['sentiments'] = sentiments

895/895 [==============================] - 17s 18ms/step


/var/folders/29/2_xxbt5d1gn7b5yqm290y2hh0000gn/T/ipykernel_22994/2579558094.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['sentiments'] = sentiments


In [19]:
df_train[['tweet_rendered_content','sentiments']].sentiments.value_counts()

positive    16145
negative    12478
Name: sentiments, dtype: int64

In [9]:
for i in df_train[df_train['sentiments'] == 'positive']['tweet_rendered_content'].iloc[:10]:
    print(i)

wasn going to tweet metoo because weve been tweeting crying hurting me too for long long time now
me seeing you post metoo several months after posting that the women march protestors were disrespectful whiners
me too metoo
metoo at work school in public
this metoo hashtag is sad smh
please tell me what you think metoo will do when the leader of our country brags about sexually assaulting women
tired of being told it didnt count because it was another woman metoo
you can report it have evidence have no fear of the judgment your rapist won go to jail don
he told me they never believe me at first they didn metoo
when was by babysitter metoo


In [10]:
df_train.to_csv('/Users/alenjose/Desktop/data/project_data/model_info/LSTM_v1_MeToo_predictions')